# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [24]:
import logging
import os
import csv
import joblib
import json
import requests
import pandas as pd 
import numpy as np 
from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
#from scripts import helpers # contains some custom functions to handle CAMLES data

## Dataset

### Overview
The primary objective was to develop an early warning system, i.e. binary classification of failed ('Target'==1) vs. survived ('Target'==0), for the US banks using their quarterly filings with the regulator. Overall, 137 failed banks and 6,877 surviving banks were used in this machine learning exercise. Historical observations from the first 4 quarters ending 2010Q3 (stored in ./data) are used to tune the model and out-of-sample testing is performed on quarterly data starting from 2010Q4 (stored in ./oos). 

### Setting up the project

In [3]:
ws = Workspace.from_config()
ws.write_config(path='.azureml')
experiment_name = 'camels-clf'
project_folder = './dmik'

exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: final-ws
Azure region: eastus
Subscription id: 0c66ad45-500d-48af-80d3-0039ebf1975e
Resource group: final-rgp


### Uploading the training dataset using GUI

In [29]:
dataset = ws.datasets['camels'] 
df = dataset.to_pandas_dataframe()

df.head()

,Column2,Target,EQTA,EQTL,LLRTA,LLRGL,OEXTA,INCEMP,ROA,ROE,TDTL,TDTA,TATA
0,1252,1,0.01,0.01,0.09,0.12,0.03,-593.17,-0.14,-15.77,1.23,0.98,0.11
1,3287,1,0.08,0.19,0.00,0.01,0.02,20.90,0.00,0.05,2.21,0.91,0.36
2,5672,1,0.00,0.00,0.07,0.10,0.03,-323.52,-0.06,-27.68,1.38,0.93,0.14
3,5702,1,0.02,0.02,0.03,0.04,0.04,-153.60,-0.05,-3.12,1.20,0.90,0.09
4,8221,1,0.01,0.01,0.04,0.05,0.04,-217.89,-0.07,-6.03,1.15,0.99,0.10


### Checking for or creating appropriate `ComputeTarget`

In [5]:
cpu_cluster_name = 'final-cmp'

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Existing compute target.')

except:
    print('Creating compute target.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

print(compute_target.get_status())

Existing compute target.
{
  "errors": [],
  "creationTime": "2021-03-22T19:24:32.034596+00:00",
  "createdBy": {
    "userObjectId": "49e75006-b9ac-415c-9176-f83c59d4bf26",
    "userTenantId": "d689239e-c492-40c6-b391-2c5951d31d14",
    "userName": null
  },
  "modifiedTime": "2021-03-22T19:27:20.575645+00:00",
  "state": "Running",
  "vmSize": "STANDARD_DS2_V2"
}


## AutoML Configuration
### Primary metric determins configuaration
Financial metrics recorded in the last reports of the failed banks should have predictive power that is needed to forecast future failures. Due to significant class imbalances and taking into account costs accosiated with financial distress, the model should aim to maximize the recall score. In other words, accuracy is probably not the best metrics, as Type II error needs to be minimized. This is why the main focus of this classification should be on maximizing AUC, hopefully, by achieving good recall score. This is why 'norm_macro_recall' was chosen as a primary metric. Timeout and number of concurrent iterations were set conservatively to control the costs.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 15,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'norm_macro_recall'
    }

automl_config = AutoMLConfig(
    compute_target=compute_target, 
    task = "classification",
    training_data=dataset, 
    label_column_name="Target", 
    path = project_folder,
    enable_early_stopping= True, 
    featurization= 'auto', 
    debug_log = "automl_errors.log",
    **automl_settings
    )

## Run Details

### Possible modeling choices 
Generally speaking, decision trees should work well for this task, as these models do not make any functional form assumptions, handle both categorical and continuous data well, and are easy to interpret. Tree-based models simply aim to reduce entropy at every split and are therefore very straightforward, no need to worry about missing data and scaling. They are not very stable though, as new data may produce a totally different tree, and they also tend to overfit.

Possible solution would be model averaging - employing “wisdom of the crowd”. It seems that for the present task two paths are possible: reducing variance or reducing bias. The former implies complex model, i.e. starting with a bushy, high-variance tree and resampling with replacement, what will produce a family of Random Forest models. The later implies starting with a simple model, i.e. possible a stump, high-bias classifier and learning from miss-classified instances, what will produce a family of Boosting models.


In [8]:
remote_run = exp.submit(config=automl_config) 
RunDetails(remote_run).show() # <--use Notebook widget
remote_run.wait_for_completion(show_output=True)

Running on remote.


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a fal

{'runId': 'AutoML_dbfdcfd4-7378-4756-b9e9-d7613df9a6be',
 'target': 'final-cmp',
 'status': 'Completed',
 'startTimeUtc': '2021-03-22T19:47:48.199852Z',
 'endTimeUtc': '2021-03-22T20:28:37.532955Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'final-cmp',
  'AMLSettingsJsonString': '{"path":null,"name":"camels-clf","subscription_id":"0c66ad45-500d-48af-80d3-0039ebf1975e","resource_group":"final-rgp","workspace_name":"final-ws","region":"eastus","compute_target":"final-cmp","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"norm_macro_recall","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featurization":"auto","

![RunDetails for AutoML](assets/automl_rundetails.png)

In [9]:
print("Run Status: ",remote_run.get_status())

Run Status:  Completed


## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
# TODO: Retrieve and save your best automl model.
best_run, fitted_model = remote_run.get_output()

print('Best run:', best_run)
print('Best model:', fitted_model)

best_run_metrics = best_run.get_metrics()

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.24.0, current version:1.22.0
Package:azureml-core, training version:1.24.0.post1, current version:1.22.0
Package:azureml-dataprep, training version:2.11.2, current version:2.9.1
Package:azureml-dataprep-native, training version:30.0.0, current version:29.0.0
Package:azureml-dataprep-rslex, training version:1.9.1, current version:1.7.0
Package:azureml-dataset-runtime, training version:1.24.0, current version:1.22.0
Package:azureml-defaults, training version:1.24.0, current version:1.22.0
Package:azureml-interpret, training version:1.24.0, current version:1.22.0
Package:azureml-mlflow, training version:1.24.0, current version:1.22.0
Package:azureml-pipeline-core, training version:1.24.0, current version:1.22.0
Package:azureml-telemetry, training version:1.24.0, current version:1.22.0
Package:azureml-train-automl-client, training version:1.24.0, current version:1.22.0
Package:azureml-train-automl-runtime, training version:1.24.0, current 

Best run: Run(Experiment: camels-clf,
Id: AutoML_dbfdcfd4-7378-4756-b9e9-d7613df9a6be_37,
Type: azureml.scriptrun,
Status: Completed)
Best model: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_samples=None,
                                                                                                    min_impurity_decrease=0.0,
 

In [12]:
fitted_model._final_estimator

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('25',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f3191d88940>),
                                                           ('logisticregression',
                                                            LogisticRegression(C=0.040949150623804234,
                                                                               class_weight='balanced',
                                                                               dual=False,
                                                                               fit_intercept=True,
                                                                               intercept_sc...
             

In [16]:
# Save the best model
joblib.dump(value=fitted_model, filename="fitted_automl_model.joblib")

['fitted_automl_model.joblib']

In [17]:
# Register the model produced by AutoML
automl_model = remote_run.register_model(model_name='automl_model.pkl') #, model_path = './outputs/')

In [18]:
# # Alt. way to save in pkl from Audrey
# os.makedirs('./models', exist_ok=True)
# remote_run.download_file('/output/model.pkl', os.path.join('models', 'automl_model'))
# # Do I need to dowload all the files? Take a look at them with
# #remote_run.get_file_names()

In [ ]:
#TODO: Save the best model
#joblib.dump(value=fitted_model, filename="fitted_automl_model.joblib") 

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# # Register the model produced by AutoML
# automl_model = remote_run.register_model(model_name='automl_model',
#                                          model_path='./outputs/model.pkl',
#                                          model_framework=Model.Framework.SCIKITLEARN,
#                                          description="Maximizing norm_macro_recall in binary classification") 

# #model = remote_run.register_model(model_name = 'house_price_model.pkl')
# #print(remote_run.model_id)

### Create inference config

In [ ]:
# Download the conda environment file produced by AutoML and create an environment <--Audrey way
#best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml') #<- consider storing environment like so?
#env = Environment.from_conda_specification(name = 'myenv', file_path = 'conda_env.yml')

In [19]:
environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)
inference_config = InferenceConfig(entry_script = entry_script, environment = environment) # <- 'env' for Aureys way

#aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)


### Deploy the model as web service

In [20]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservice", [automl_model], inference_config, deployment_config) #<=aci-_config in Audrey
service.wait_for_deployment(show_output = True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


Check the state of the service

In [38]:
print("Checking service status:{}".format(service.state))

Checking service - status: Healthy


If 'Healthy', get URIs

In [40]:
print("Scoring URI: {}".format(service.scoring_uri))
print("Swagger URI: {}".format(service.swagger_uri))

Scoring URI: http://e6d3ccd2-b44e-4259-87fe-8bdc28dff4ec.eastus.azurecontainer.io/score
Swagger URI: http://e6d3ccd2-b44e-4259-87fe-8bdc28dff4ec.eastus.azurecontainer.io/swagger.json


In [44]:
pkey, skey = service.get_keys()
print("primary key: {},\nsecond key: {}".format(pkey, skey)) 

primary key: taQRhOtWcivd0YtpegeGV1w9r77N0BT7,
second key: t2ra1FhQWOBjWebWU5EiU80S2uB35K9a


Take a small sample of feautures of the failed banks (`'Target'=1`)

In [47]:
sample = df.loc[df['Target']==1].sample(10)
y = sample.pop('Target')

In [49]:
# convert the sample records to a json data file
scoring_json = json.dumps({'data': sample.to_dict(orient='records')})
print(f'{scoring_json}')

# Set the content type
headers = {"Content-Type": "application/json"}

# set the authorization header
headers["Authorization"] = f"Bearer {pkey}"

# post a request to the scoring uri
resp = requests.post(service.scoring_uri, scoring_json, headers=headers)

# print the scoring results
print(resp.json())

# compare the scoring results with the corresponding y label values
print(f'True Values: {y.values}')

{"data": [{"Column2": "57735", "EQTA": -0.007285795965041248, "EQTL": -0.007583744261180071, "LLRTA": 0.08465245446377521, "LLRGL": 0.08811426628124469, "OEXTA": 0.011173731928448909, "INCEMP": -100.41666666666667, "ROA": -0.019684717797925345, "ROE": 2.701793721973094, "TDTL": 0.9911239585104574, "TDTA": 0.9521849219962427, "TATA": 0.016335865392469166}, {"Column2": "35061", "EQTA": 0.027414691804343545, "EQTL": 0.03752095022954165, "LLRTA": 0.02661072628434823, "LLRGL": 0.036420607738832614, "OEXTA": 0.01689392446983532, "INCEMP": -43.35, "ROA": -0.009232292792528978, "ROE": -0.3367644202757817, "TDTL": 1.1994024630182905, "TDTA": 0.8763437138947604, "TATA": 0.05075631325904195}, {"Column2": "30600", "EQTA": 0.014721146039934642, "EQTL": 0.017952477629896983, "LLRTA": 0.023245537411750197, "LLRGL": 0.028347996089931573, "OEXTA": 0.011335025537206219, "INCEMP": -19.891304347826086, "ROA": -0.004701517845214729, "ROE": -0.3193717277486911, "TDTL": 1.0190427851718173, "TDTA": 0.83562157

In [51]:
# another way to test the scoring uri
print("Prediction: {}".format(service.run(scoring_json)))
print(f'True Values: {y.values}')

Prediction: {"result": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
True Values: [1 1 1 1 1 1 1 1 1 1]


TODO: In the cell below, print the logs of the web service and delete the service

In [33]:
# print the logs by calling the get_logs() function of the web service
print(f'webservice logs: \n{service.get_logs()}\n')

webservice logs: 
2021-03-22T21:06:32,392545800+00:00 - iot-server/run 
2021-03-22T21:06:32,393517400+00:00 - gunicorn/run 
2021-03-22T21:06:32,423816600+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-03-22T21:06:32,432858200+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sb

## Clean Up

In [ ]:
service.delete()

In [ ]:
sevice.state()